In [51]:
import pandas as pd
from collections import Counter
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from sklearn.feature_extraction import text

In [90]:
data = pd.read_pickle('pickled_data/for_word_cloud.pkl')
#data.drop(['President', 'Party','speech', 'first_clean_tokenized'], axis=1, inplace = True)
data = data.rename({'first_clean' : 'speech'}, axis=1)
#data =data.set_index('year')
years = data['year'].to_list()

print(years)
print(data.head())
print(data.dtypes)


['1926', '1927', '1928', '1930', '1931', '1932', '1986', '1987', '1988', '1992', '1993', '1994', '2005', '2006', '2007', '2008', '2009', '2010']
   year                                             speech
0  1926  members of the congress: in reporting to the c...
1  1927  members of the congress: it is gratifying to r...
2  1928  to the congress of the united states:no congre...
3  1930  to the senate and house of representatives: i ...
4  1931  to the senate and house of representatives: it...
year      object
speech    object
dtype: object


In [91]:
stop_words = text.ENGLISH_STOP_WORDS

wc = WordCloud(stopwords=stop_words, background_color="white", colormap="Dark2",
               max_font_size=150, random_state=42)

In [92]:
data

,year,speech
0,1926,members of the congress: in reporting to the c...
1,1927,members of the congress: it is gratifying to r...
2,1928,to the congress of the united states:no congre...
3,1930,to the senate and house of representatives: i ...
4,1931,to the senate and house of representatives: it...
5,1932,to the senate and house of representatives: in...
6,1986,"mr. speaker, mr. president, distinguished memb..."
7,1987,"thank you very much. mr. speaker, mr. presiden..."
8,1988,"""thank you. mr. speaker, mr. president, and di..."
9,1992,"mr. speaker and mr. president, distinguished m..."


In [95]:
for column in data:
      
    # Select column contents by column
    # name using [] operator
    columnSeriesObj = data.speech
    print('Colunm Name : ', speech)
    print('Column Contents : ', columnSeriesObj.values)
    print('Column length: ', len(columnSeriesObj.values))

Colunm Name :  first_clean
Column Contents :  ["members of the congress: in reporting to the congress the state of the union, i find it impossible to characterize it other than one of general peace and prosperity. in some quarters our diplomacy is vexed with difficult and as yet unsolved problems, but nowhere are we met with armed conflict. if some occupations and areas are not flourishing, in none does there remain any acute chronic depression. what the country requires is not so much new policies as a steady continuation of those which are already being crowned with such abundant success. it can not be too often repeated that in common with all the world we are engaged in liquidating the war. in the present short session no great amount of new legislation is possible, but in order to comprehend what is most desirable some survey of our general situation is necessary. a large amount of time is consumed in the passage of appropriation bills. if each congress in its opening session woul

In [88]:
data
plt.rcParams['figure.figsize'] = [30, 15]


for index, sp in enumerate(data.columns):
    wc.generate(data.speech[sp])
    
    plt.subplot(3, 4, index+1)
    plt.imshow(wc, interpolation="bilinear")
    plt.axis("off")
    plt.title(years[index])
    
plt.show()

KeyError: 'year'